Choosing and checking our GPU used.

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:

pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 526MB/s]


#####Albumentations

In [ ]:
import albumentations as A
import cv2
import os
import shutil
import yaml
from pathlib import Path
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm

# Define simple transforms for smoke/fire detection
def get_simple_transforms():
    return A.Compose([
        # Essential color transforms for smoke/fire
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.6
        ),
        A.HueSaturationValue(
            hue_shift_limit=10,
            sat_shift_limit=20,
            val_shift_limit=10,
            p=0.5
        ),

        # Blur for smoke simulation
        A.GaussianBlur(blur_limit=(3, 5), p=0.3),

        # Basic geometric
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=10, p=0.3),

        # Resize to ensure consistent input size
        A.Resize(640, 640, p=1.0),

    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def augment_single_image(image_path, label_path, output_dir, transform, num_augmentations=2):
    """
    Augment a single YOLO format image and label pair
    """
    try:
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image: {image_path}")
            return

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read YOLO format labels
        bboxes = []
        class_labels = []

        if os.path.exists(label_path) and os.path.getsize(label_path) > 0:
            with open(label_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

        # If no bboxes, still augment the image (background images)
        if not bboxes:
            bboxes = []
            class_labels = []

        # Create augmented versions
        base_name = Path(image_path).stem

        for i in range(num_augmentations):
            try:
                transformed = transform(
                    image=image,
                    bboxes=bboxes,
                    class_labels=class_labels
                )

                aug_image = transformed['image']
                aug_bboxes = transformed['bboxes']
                aug_labels = transformed['class_labels']

                # Save augmented image
                aug_image_path = os.path.join(output_dir, 'images', f"{base_name}_aug_{i}.jpg")
                os.makedirs(os.path.dirname(aug_image_path), exist_ok=True)
                aug_image_bgr = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(aug_image_path, aug_image_bgr)

                # Save augmented labels
                aug_label_path = os.path.join(output_dir, 'labels', f"{base_name}_aug_{i}.txt")
                os.makedirs(os.path.dirname(aug_label_path), exist_ok=True)

                with open(aug_label_path, 'w') as f:
                    for bbox, label in zip(aug_bboxes, aug_labels):
                        f.write(f"{label} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

            except Exception as e:
                print(f"Error creating augmentation {i} for {image_path}: {e}")
                continue

    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def augment_dataset(original_dataset_path, augmented_dataset_path, num_augmentations=2):
    """
    Augment entire YOLO dataset
    """
    print("Starting dataset augmentation...")

    # Create output directories
    os.makedirs(augmented_dataset_path, exist_ok=True)

    # Get transform
    transform = get_simple_transforms()

    # Process each split (train, val, test)
    for split in ['train', 'val', 'test']:
        print(f"\nProcessing {split} split...")

        # Create directories
        split_images_dir = os.path.join(augmented_dataset_path, split, 'images')
        split_labels_dir = os.path.join(augmented_dataset_path, split, 'labels')
        os.makedirs(split_images_dir, exist_ok=True)
        os.makedirs(split_labels_dir, exist_ok=True)

        # Source directories
        src_images_dir = os.path.join(original_dataset_path, split, 'images')
        src_labels_dir = os.path.join(original_dataset_path, split, 'labels')

        if not os.path.exists(src_images_dir):
            print(f"Source directory not found: {src_images_dir}")
            continue

        # Get all image files
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp']:
            image_files.extend(Path(src_images_dir).glob(ext))

        print(f"Found {len(image_files)} images in {split}")

        # First, copy original files
        print(f"Copying original {split} files...")
        for img_path in tqdm(image_files):
            # Copy image
            dst_img = os.path.join(split_images_dir, img_path.name)
            shutil.copy2(img_path, dst_img)

            # Copy label if exists
            label_path = os.path.join(src_labels_dir, img_path.stem + '.txt')
            if os.path.exists(label_path):
                dst_label = os.path.join(split_labels_dir, img_path.stem + '.txt')
                shutil.copy2(label_path, dst_label)
            else:
                # Create empty label file
                dst_label = os.path.join(split_labels_dir, img_path.stem + '.txt')
                open(dst_label, 'w').close()

        # Then create augmented versions (only for training set)
        if split == 'train':
            print(f"Creating {num_augmentations} augmentations per training image...")
            for img_path in tqdm(image_files):
                label_path = os.path.join(src_labels_dir, img_path.stem + '.txt')
                augment_single_image(
                    str(img_path),
                    label_path,
                    os.path.join(augmented_dataset_path, split),
                    transform,
                    num_augmentations
                )

    print("Dataset augmentation completed!")

def create_augmented_data_yaml(original_yaml_path, augmented_dataset_path, output_yaml_path):
    """
    Create data.yaml file for augmented dataset
    """
    # Read original yaml
    with open(original_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # Update paths
    data['train'] = os.path.join(augmented_dataset_path, 'train', 'images')
    data['val'] = os.path.join(augmented_dataset_path, 'val', 'images')
    data['test'] = os.path.join(augmented_dataset_path,'test', 'images')

    # Save new yaml
    with open(output_yaml_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)

    print(f"Created augmented data.yaml at: {output_yaml_path}")
    return output_yaml_path

def train_with_albumentations():
    """
    Complete training pipeline with Albumentations
    """
    # Paths
    original_dataset = "/content/dataset"
    augmented_dataset = "/content/augmented_dataset"
    original_yaml = "/content/dataset/data.yaml"
    augmented_yaml = "/content/augmented_dataset/data.yaml"

    print("=== YOLO Training with Albumentations ===")
    print(f"Original dataset: {original_dataset}")
    print(f"Augmented dataset: {augmented_dataset}")

    # Step 1: Create augmented dataset
    if not os.path.exists(augmented_dataset):
        print("\nStep 1: Creating augmented dataset...")
        augment_dataset(original_dataset, augmented_dataset, num_augmentations=2)
    else:
        print("\nStep 1: Augmented dataset already exists, skipping...")

    # Step 2: Create data.yaml for augmented dataset
    print("\nStep 2: Creating data.yaml for augmented dataset...")
    create_augmented_data_yaml(original_yaml, augmented_dataset, augmented_yaml)

    # Step 3: Train baseline model (with built-in augmentations)
    # print("\nStep 3: Training baseline model with YOLO built-in augmentations...")
    # baseline_model = YOLO("yolov8s.pt")

    # baseline_results = baseline_model.train(
    #     data=original_yaml,
    #     epochs=200,  # Reduced for comparison
    #     batch=32,
    #     augment=True,  # YOLO built-in augmentations
    #     patience=15,
    #     name="baseline_builtin",
    #     verbose=True
    # )

    # print(f"Baseline training completed!")
    # print(f"Baseline mAP50: {baseline_results.results_dict.get('metrics/mAP50(B)', 'N/A')}")

    # Step 4: Train with Albumentations-augmented dataset
    print("\nStep 4: Training with Albumentations-augmented dataset...")
    albumentations_model = YOLO("/content/runs/detect/baseline_builtin2/weights/best.pt")  # Start fresh or use best baseline

    albumentations_results = albumentations_model.train(
        data=augmented_yaml,
        epochs=200,
        batch=32,
        augment=False,  # Disable built-in since we pre-augmented
        patience=15,
        name="albumentations_augmented",
        verbose=True
    )

    print(f"Albumentations training completed!")
    print(f"Albumentations mAP50: {albumentations_results.results_dict.get('metrics/mAP50(B)', 'N/A')}")

    # Step 5: Compare results
    print("\n=== RESULTS COMPARISON ===")
    # baseline_map = baseline_results.results_dict.get('metrics/mAP50(B)', 0)
    albumentations_map = albumentations_results.results_dict.get('metrics/mAP50(B)', 0)

    # print(f"Baseline (YOLO built-in):     mAP50 = {baseline_map:.3f}")
    print(f"Albumentations (pre-augmented): mAP50 = {albumentations_map:.3f}")

    # if albumentations_map > baseline_map:
    #     improvement = albumentations_map - baseline_map
    #     print(f"✅ Improvement: +{improvement:.3f} mAP50 ({improvement/baseline_map*100:.1f}%)")
    # else:
    #     decrease = baseline_map - albumentations_map
    #     print(f"❌ Decrease: -{decrease:.3f} mAP50 ({decrease/baseline_map*100:.1f}%)")

    return albumentations_results

# Quick test function
def test_transforms():
    """
    Test that transforms work correctly
    """
    print("Testing Albumentations transforms...")

    transform = get_simple_transforms()

    # Create dummy data
    dummy_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
    dummy_bboxes = [[0.5, 0.5, 0.2, 0.3], [0.3, 0.7, 0.1, 0.1]]  # YOLO format
    dummy_labels = [0, 1]  # smoke, fire

    try:
        result = transform(
            image=dummy_image,
            bboxes=dummy_bboxes,
            class_labels=dummy_labels
        )

        print("✅ Transform test successful!")
        print(f"Input image shape: {dummy_image.shape}")
        print(f"Output image shape: {result['image'].shape}")
        print(f"Input bboxes: {dummy_bboxes}")
        print(f"Output bboxes: {result['bboxes']}")
        print(f"Labels preserved: {result['class_labels']}")

        return True

    except Exception as e:
        print(f"❌ Transform test failed: {e}")
        return False

if __name__ == "__main__":
    # First test transforms
    if test_transforms():
        print("\n" + "="*50)
        # Run full training pipeline
        baseline_results, albumentations_results = train_with_albumentations()
    else:
        print("Fix transform issues before proceeding with training.")

Testing Albumentations transforms...
✅ Transform test successful!
Input image shape: (480, 640, 3)
Output image shape: (640, 640, 3)
Input bboxes: [[0.5, 0.5, 0.2, 0.3], [0.3, 0.7, 0.1, 0.1]]
Output bboxes: [[0.49999998435732645, 0.49999998651740485, 0.2151710023059915, 0.31782780796162585], [0.7099523981048264, 0.6809603177099683, 0.10497623033776027, 0.10903512280268457]]
Labels preserved: [0.0, 1.0]

=== YOLO Training with Albumentations ===
Original dataset: /content/dataset
Augmented dataset: /content/augmented_dataset

Step 1: Augmented dataset already exists, skipping...

Step 2: Creating data.yaml for augmented dataset...
Created augmented data.yaml at: /content/augmented_dataset/data.yaml

Step 4: Training with Albumentations-augmented dataset...
New https://pypi.org/project/ultralytics/8.3.171 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, 

FileNotFoundError: [34m[1mtrain: [0mError loading data from /content/augmented_dataset/train/images
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
!yolo task=detect mode=train data="/content/FireSmokeDataset-3/data.yaml" model=yolov8s.pt epochs=150 batch=32 imgsz=640 optimizer="SGD" lr0=0.01 momentum=0.937 weight_decay=0.0005 augment=True hsv_h=0.03 hsv_s=0.7 patience=20

100% 21.5M/21.5M [00:00<00:00, 186MB/s]
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/FireSmokeDataset-3/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.03, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, pati

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="aAbQRSqmt35Aah8o74ur")
project = rf.workspace("catargiuconstantin").project("firesmokedataset")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to FireSmokeDataset-3 in yolov8:: 100%|██████████| 93138/93138 [00:13<00:00, 7030.14it/s] 


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/dataset.zip"
extract_to = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


In [ ]:
!cp -r /content/runs /content/drive/MyDrive/